# Config

> Esta seção tem configurações gerais do projeto, como importações de pacotes, carregamento de variáveis de ambiente entre outros.

In [1]:
# Importação de módulos.
import requests
import pandas as pd
import warnings
import json
import yaml
import gzip
import os
import glob
import importlib
#from sqlalchemy import create_engine, inspect, text
#from sqlalchemy import create_engine
from datetime import datetime, timedelta
from work.bdt_data_integration.src.utils import Utils, WebhookNotifier, DiscordNotifier
from work.bdt_data_integration.src.writers import DataWriter
from work.bdt_data_integration.src.streams import NotionApiStream
from work.bdt_data_integration.src.loaders import PostgresLoader
from work.bdt_data_integration.src.transformers import NotionTransformer

In [2]:
source = 'notion'

# Carregamento do arquivo config.yaml
config = Utils.load_config()

# Carregamento das variáveis de ambiente
api_key = os.getenv('NOTION_APIKEY')
database_id = os.getenv('UTD_DATABASE_ID')
host = os.environ["NEON_HOST"]
user =os.environ["NEON_ROOT_USER"]
password = os.environ["NEON_ROOT_PASSWORD"]
db_name = os.environ["NEON_DATABASE_NAME"]
discord_token = os.getenv('DISCORD_TOKEN')
discord_channel = os.getenv('DISCORD_CHANNEL')
notifier_url = os.getenv('MAKE_NOTIFICATION_WEBHOOK')

In [3]:
notifier = WebhookNotifier(url=notifier_url,pipeline=f'{source}_pipeline')
notifier.pipeline_start()

Accepted


# Extract

> Nesta seção é realizada a extração de dados brutos da API, são instanciados uma stream e um writer, que serão responsáveis pela conexão com a fonte de dados e com a escrita dos dados, respectivamente.

In [4]:
try:
    writer = DataWriter(
        source=source,
        stream='universal_task_database', 
        compression = True,
        config=config
        )
except Exception as e:
    notifier.pipeline_error(e)
    raise e

> Usa-se o parâmetro compression = True para que a saída da stream aconteça em formato gzip, para economizar espaço.

In [5]:
# Instanciar Stream para a extração dos dados
try:
    stream = NotionApiStream(
        identifier = source,
        base_endpoint = 'https://api.notion.com/v1',
        token = api_key,
        auth_method = 'bearer',
        database_id = database_id,
        writer = writer)
except Exception as e:
    notifier.pipeline_error(e)
    raise e

In [6]:
#Executar a extração dos dados
try:
    records, time = stream.run()
except Exception as e:
    notifier.pipeline_error(e)
    raise e

INFO:work.bdt_data_integration.src.streams:Attempting scroll fetch from https://api.notion.com/v1/databases/3290a9e9f0bf4d84bc57aae53f635e7e/query
INFO:work.bdt_data_integration.src.streams:Gettinng data from endpoint https://api.notion.com/v1/databases/3290a9e9f0bf4d84bc57aae53f635e7e/query
INFO:work.bdt_data_integration.src.streams:Successfully fetched page 1
INFO:work.bdt_data_integration.src.streams:Gettinng data from endpoint https://api.notion.com/v1/databases/3290a9e9f0bf4d84bc57aae53f635e7e/query
INFO:work.bdt_data_integration.src.streams:Successfully fetched page 2
INFO:work.bdt_data_integration.src.streams:Gettinng data from endpoint https://api.notion.com/v1/databases/3290a9e9f0bf4d84bc57aae53f635e7e/query
INFO:work.bdt_data_integration.src.streams:Successfully fetched page 3
INFO:work.bdt_data_integration.src.streams:Gettinng data from endpoint https://api.notion.com/v1/databases/3290a9e9f0bf4d84bc57aae53f635e7e/query
INFO:work.bdt_data_integration.src.streams:Successfully 

# Transform

> Nesta etapa ocorre a transformação dos dados, como obtenção de chaves e valores, alteração da estrutura, mudanças no texto e aí por diante.

In [7]:
# Instanciar tranformador de dados
try:
    transformer = NotionTransformer()
except Exception as e:
    notifier.pipeline_error(e)
    raise e

## ntn_utd_pages

### processing

In [8]:
try:
    file_path = Utils.get_latest_file('/work/data/raw/notion/universal_task_database', '.txt.gz')
    if file_path:
        records = Utils.read_records(file_path)
    else:
        print('No files found in the specified directory')
except Exception as e:
    notifier.pipeline_error(e)
    raise e

In [9]:
try:
    # Obter Dataframe com propriedades extraídas
    extracted_properties_df = transformer.extract_pages_from_records(records)

    # Converter todas as colunas de lista do dataframe para strings separadas por vírgulas
    transformer.process_list_columns(extracted_properties_df)

    # Converter todas as colunas de lista do dataframe para strings separadas por vírgulas
    extracted_properties_df['Etapa'] = extracted_properties_df['Etapa'].str[4:]

    # Atualizar a coluna Task Interval com o atributo 'start' do objeto
    extracted_properties_df['Task Interval']  = extracted_properties_df['Task Interval'].apply(lambda x: x['start'] if isinstance(x, dict) and 'start' in x else None)

    # Definindo uma coluna para identificar o horário da atualização

    extracted_properties_df['loaded_at'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    extracted_properties_df['extracted_at'] = time

    extracted_properties_df
except Exception as e:
    notifier.pipeline_error(e)
    raise e

In [10]:
try:
    output = writer.get_output_file_path('notion', 'ntn__utd_pages',target_layer='processing') + '.csv'
    os.makedirs(os.path.dirname(output), exist_ok=True)
    extracted_properties_df.to_csv(output, index=False)
except Exception as e:
    notifier.pipeline_error(e)
    raise e

### staging

In [11]:
ntn__utd_pages = (lambda: _deepnote_execute_sql('SELECT id AS page_id,\n       "Task ID" as task_id,\n--       extracted_at as __deepnote_extracted_at,\n--       loaded_at as __deepnote_loaded_at,\n       Branch as branch,\n       archived,\n       in_trash,\n       last_edited_time,\n       created_time,\n       "Título" as titulo,\n       "Created At" as created_at,\n       "Created By" as created_by,\n       "Updated At" as updated_at,\n       "Updated By" as updated_by,\n       "Responsável" as responsavel,\n       "Solicitante" as solicitante,\n       "Tester" as tester,\n       "Área" as area,\n       "Ambiente" as ambiente,\n       "Prioridade" as prioridade,\n       "SLA" as sla,\n       "Etapa" as etapa,\n       "Dias na Etapa" as dias_na_etapa,\n       "Dias até o SLA" as dias_ate_o_sla,\n       "Dias até o Desenvolvimento" as dias_ate_o_desenvolvimento,\n       "Dias de Desenvolvimento" as dias_de_desenvolvimento,\n       "Customer ID" as customer_id,\n       "Nome do Cliente" as nome_do_cliente,\n       "Justificativa" as justificativa,\n       "Sprint" as sprint,\n       "Bloqueando" as bloqueando,\n       "Bloqueado por" as bloqueado_por,\n       "Integração" as integracao,\n       "Projetos" as projetos,\n       "Stage Updated At" as stage_updated_at,\n       "Entered Approved At" as entered_approved_at,\n       "Entered Awaiting Version At" as entered_awaiting_version_at,\n       "Entered Backlog At" as entered_backlog_at,\n       "Entered Correction At" as entered_correction_at,\n       "Entered Doing At" as entered_doing_at,\n       "Entered Done At" as entered_done_at,\n       "Entered Testing At" as entered_testing_at,\n       "Entered To-Do At" as entered_to_do_at,\n       "Entered To-Test At" as entered_to_test_at,\n       "OKR" as okr\nFROM extracted_properties_df;', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled') if '_deepnote_execute_sql' in globals() else _dntk.execute_sql('SELECT id AS page_id,\n       "Task ID" as task_id,\n--       extracted_at as __deepnote_extracted_at,\n--       loaded_at as __deepnote_loaded_at,\n       Branch as branch,\n       archived,\n       in_trash,\n       last_edited_time,\n       created_time,\n       "Título" as titulo,\n       "Created At" as created_at,\n       "Created By" as created_by,\n       "Updated At" as updated_at,\n       "Updated By" as updated_by,\n       "Responsável" as responsavel,\n       "Solicitante" as solicitante,\n       "Tester" as tester,\n       "Área" as area,\n       "Ambiente" as ambiente,\n       "Prioridade" as prioridade,\n       "SLA" as sla,\n       "Etapa" as etapa,\n       "Dias na Etapa" as dias_na_etapa,\n       "Dias até o SLA" as dias_ate_o_sla,\n       "Dias até o Desenvolvimento" as dias_ate_o_desenvolvimento,\n       "Dias de Desenvolvimento" as dias_de_desenvolvimento,\n       "Customer ID" as customer_id,\n       "Nome do Cliente" as nome_do_cliente,\n       "Justificativa" as justificativa,\n       "Sprint" as sprint,\n       "Bloqueando" as bloqueando,\n       "Bloqueado por" as bloqueado_por,\n       "Integração" as integracao,\n       "Projetos" as projetos,\n       "Stage Updated At" as stage_updated_at,\n       "Entered Approved At" as entered_approved_at,\n       "Entered Awaiting Version At" as entered_awaiting_version_at,\n       "Entered Backlog At" as entered_backlog_at,\n       "Entered Correction At" as entered_correction_at,\n       "Entered Doing At" as entered_doing_at,\n       "Entered Done At" as entered_done_at,\n       "Entered Testing At" as entered_testing_at,\n       "Entered To-Do At" as entered_to_do_at,\n       "Entered To-Test At" as entered_to_test_at,\n       "OKR" as okr\nFROM extracted_properties_df;', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled'))()
ntn__utd_pages

,page_id,task_id,branch,archived,in_trash,last_edited_time,created_time,titulo,created_at,created_by,...,entered_approved_at,entered_awaiting_version_at,entered_backlog_at,entered_correction_at,entered_doing_at,entered_done_at,entered_testing_at,entered_to_do_at,entered_to_test_at,okr
0,df559ffe-26f0-464d-9891-b9b7fea94190,HF-1015,,False,False,2024-09-10T18:11:00.000Z,2024-09-09T18:54:00.000Z,#1015 - Loading infinito APP,2024-09-09T18:54:00.000Z,Cristian Kestring,...,None,None,None,None,None,None,None,None,None,None
1,3a1790ae-5f03-4cec-abf5-224271f76dad,PRD-1388,,False,False,2024-09-09T18:24:00.000Z,2024-09-09T18:20:00.000Z,Exibir data da última atualização do produto n...,2024-09-09T18:20:00.000Z,Cauê Ausec,...,None,None,None,None,None,None,None,None,None,None
2,1e15cafa-1b44-4a91-8738-28793c82ad26,PRD-1384,,False,False,2024-09-04T18:03:00.000Z,2024-09-04T16:57:00.000Z,O botão “Ver detalhes” na pagina de pedidos em...,2024-09-04T16:57:00.000Z,Fernando Luppi,...,None,None,None,None,None,None,None,None,None,None
3,74915308-d1a4-4403-b3f9-431d2d6ec9aa,HF-1014,,False,False,2024-09-10T22:07:00.000Z,2024-09-04T16:55:00.000Z,#1014 Tiny - Duplicação de Contato ao Importar...,2024-09-04T16:55:00.000Z,Laura Prass,...,None,None,2024-09-06T13:57:00.000-03:00,None,2024-09-10T17:46:00.000-03:00,None,None,2024-09-10T17:46:00.000-03:00,None,None
4,4f76e0c7-d2df-40da-b1df-17959f63e58f,PRD-1382,PRD-1382,False,False,2024-09-10T16:48:00.000Z,2024-09-03T16:59:00.000Z,PRD-1382 - Incluir atualização parcial de prod...,2024-09-03T16:59:00.000Z,Cauê Ausec,...,2024-09-09T15:38:00.000-03:00,2024-09-09T15:46:00.000-03:00,2024-09-03T13:59:00.000-03:00,2024-09-10T11:16:00.000-03:00,2024-09-05T13:33:00.000-03:00,None,2024-09-09T15:55:00.000-03:00,2024-09-05T13:32:00.000-03:00,2024-09-06T12:14:00.000-03:00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,d38a422c-b44d-4027-b208-d4e7d6cacf7d,HF-264,,False,False,2024-05-14T13:31:00.000Z,2024-03-01T18:26:00.000Z,#264 Não está aplicando valor de desconto por ...,2024-03-01T18:26:00.000Z,Cauê Ausec,...,None,None,None,None,None,None,None,None,None,None
1190,dd81b65a-c1c5-4e9e-83d5-7a812132e81e,HF-352,,False,False,2024-04-11T18:30:00.000Z,2024-03-01T18:26:00.000Z,#352 - Erro na importação do Bling,2024-03-01T18:26:00.000Z,Cauê Ausec,...,None,None,None,None,None,None,None,None,None,None
1191,e45cd20b-8bfb-4eec-9637-8231ca90dbba,HF-606,,False,False,2024-04-11T18:30:00.000Z,2024-03-01T18:26:00.000Z,#606 - Problemas na importação de clientes Omie,2024-03-01T18:26:00.000Z,Cauê Ausec,...,None,None,None,None,None,None,None,None,None,None
1192,31a3ea18-814b-4e00-91da-02770e84df1d,HF-523,,False,False,2024-04-11T18:30:00.000Z,2024-03-01T18:26:00.000Z,#523 - APP criando pedidos duplicados e subind...,2024-03-01T18:26:00.000Z,Cauê Ausec,...,None,None,None,None,None,None,None,None,None,None


In [12]:
try:
    output = writer.get_output_file_path('notion', 'ntn__utd_pages',target_layer='staging') + '.csv'
    os.makedirs(os.path.dirname(output), exist_ok=True)
    ntn__utd_pages.to_csv(output, index=False)
except Exception as e:
    notifier.pipeline_error(e)
    raise e

## ntn_utd_users

### processing

In [13]:
try:
    file_path = Utils.get_latest_file('/work/data/raw/notion/universal_task_database', '.txt.gz')
    if file_path:
        records = Utils.read_records(file_path)
    else:
        print('No files found in the specified directory')
except Exception as e:
    notifier.pipeline_error(e)
    raise e

In [14]:
try:
    ntn__utd_users = transformer._extract_users_list(records)
    output = writer.get_output_file_path('notion', 'ntn__utd_users',target_layer='processing') + '.csv'
    os.makedirs(os.path.dirname(output), exist_ok=True)
    ntn__utd_users.to_csv(output, index=False)
except Exception as e:
    notifier.pipeline_error(e)
    raise e

### staging

In [15]:
try:
    # Carregar os arquivos em processing
    ntn__utd_users = pd.read_csv("data/processing/notion/ntn__utd_users.csv")
    ntn__utd_users
except Exception as e:
    notifier.pipeline_error(e)
    raise e

In [16]:
try:
    # Gravar os arquivos em staging.
    output = writer.get_output_file_path('notion', 'ntn__utd_users',target_layer='staging') + '.csv'
    os.makedirs(os.path.dirname(output), exist_ok=True)
    ntn__utd_users.to_csv(output, index=False)
except Exception as e:
    notifier.pipeline_error(e)
    raise e

# Load

In [17]:
# Correcting the instantiation of the PostgresLoader by ensuring parameter names match expected signature
try:
    loader = PostgresLoader(user=user, password=password, host=host, db_name=db_name)
except Exception as e:
    notifier.pipeline_error(e)
    raise e

## ntn_utd_pages

In [18]:
try:
    ntn__utd_pages = pd.read_csv("data/staging/notion/ntn__utd_pages.csv")
    loader.load_data(dataframe=ntn__utd_pages, target_table='ntn__utd_pages', mode='replace', target_schema='public')
except Exception as e:
    notifier.pipeline_error(e)
    raise e

## ntn_utd_users

In [19]:
try:
    # Carregar o arquivo ntn utd users em um dataframe e então enviar para o PostgreSQL.
    ntn__utd_users = pd.read_csv("data/staging/notion/ntn__utd_users.csv")
    loader.load_data(dataframe=ntn__utd_users, target_table='ntn__utd_users', mode='replace', target_schema='public')
except Exception as e:
    notifier.pipeline_error(e)
    raise e

# Callbacks

In [20]:
notifier.pipeline_end()

Accepted


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=022588a7-e3ac-4acd-8e50-3c39b9590c40' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>